# Dự báo Định lượng Trễ Tàu — Regression Pipeline (Tối ưu RMSE)

Mục tiêu: Xây dựng pipeline hồi quy dự đoán `DELAY_MINUTES` tối ưu cho RMSE. Notebook bao gồm: tiền xử lý dữ liệu, EDA, feature engineering (lag, rolling mean, cyclical time), validation thời gian, baseline, RandomForest, XGBoost/LightGBM, tuning và lưu mô hình.

Paths (Được sử dụng trong notebook):
- DATATRAIN = `D:\MSE\5. Data Mining\railway-delay\data\processed\merged_train_data.csv`
- DATATEST = `D:\MSE\5. Data Mining\railway-delay\data\raw\railway-delay-dataset.csv`

> Lưu ý: Notebook được thiết kế để chạy trên dữ liệu đầy đủ, nhưng nếu bộ nhớ hệ thống hạn chế, hãy bật tùy chọn `DOWNSAMPLE = True` để thực hiện thử nghiệm nhanh.

## 1. Giới thiệu tổng quan

Bài toán: Dự đoán độ trễ (`DELAY_MINUTES`) để hỗ trợ tối ưu vận hành, giảm thiểu rủi ro và nâng cao trải nghiệm khách hàng. Mục tiêu chính là giảm RMSE trên tập kiểm thử bằng pipeline: Tiền xử lý, tạo feature, training (XGBoost / LightGBM), tuning, và triển khai.

Tóm tắt các bước trong notebook:
1. Setup môi trường, đọc dữ liệu.
2. Load và kiểm tra nhanh dữ liệu.
3. Tiền xử lý (missing values, loại bỏ outliers, winsorization, chuẩn hóa).
4. EDA: Histogram/KDE, Boxplot, Heatmap, phân tích theo thời gian.
5. Tạo feature: cyclical time, lag, rolling mean, tương tác (weather, distance*stops).
6. Training: baseline (mean, linear), random forest, XGBoost/LightGBM; TimeSeriesSplit cho validation.
7. So sánh mô hình và kết luận (RMSE là chỉ số chính).

Lưu ý: Nếu dataset lớn, bật `DOWNSAMPLE = True` để thử nghiệm nhanh trước khi chạy full pipeline.

## Workflow Plan

This notebook follows the following structure to operationalize railway delay prediction:

1. **Introduction** (background, motivations, and project objectives).
2. **Data Description** (dataset overview and metadata, ensuring we understand inputs before modeling).
3. **Data Preprocessing** (missing handling, encoding, feature creation, scaling, and train/test splitting).
4. **Exploratory Data Analysis** (distributions, correlations, temporal patterns, and PCA to guide modeling choices).
5. **New Features & Evaluation Metrics** (proposed engineered features plus balanced/ROC/F2 metrics to handle imbalance).
6. **Model Training & Evaluation** (baselines, tree ensembles, optional SVM/KNN/Naive Bayes, time-aware CV, and tuning).
7. **Comparison & Recommendations** (contrast models with past results, record improvements, highlight SHAP insights, and suggest deployment next steps).

Each subsequent section in the notebook aligns with this plan and records its outputs in the shared `RESULTS` log for easy comparison.

In [ ]:
# Section 1: Setup
import os
import sys
import time
from datetime import datetime
import os
print(os.getcwd())

# Add project root to sys.path to import custom modules
import sys
sys.path.append(r"D:\AnDB\L\mse\railway-delayy")

# Import helper functions
from src.utils.feature_helpers import compute_prev_delay_safe, compute_rolling_features_safe

# Paths
DATATRAIN = r"D:\AnDB\L\mse\railway-delay\docs\merged_train_data.csv"
DATATEST = r"D:\AnDB\L\mse\railway-delay\docs\merged_train_data.csv"
MODEL_DIR = os.path.join(os.getcwd(), "models")
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# Reproducibility
RANDOM_STATE = 42
import numpy as np
np.random.seed(RANDOM_STATE)

# Dependency imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Utility to install and import packages in notebook reliably
import importlib
import subprocess

# Toggle to allow the notebook to pip-install optional packages at runtime (opt-in)
AUTO_INSTALL = True

# Ensure secure and controlled install behavior
def install_and_import(pkg_name, import_name=None, extras=None):
    import_name = import_name or pkg_name
    try:
        return importlib.import_module(import_name)
    except Exception:
        if not AUTO_INSTALL:
            print(f"Package {pkg_name} not found and AUTO_INSTALL is False. Skipping install.")
            return None
        try:
            pkg_install = pkg_name if extras is None else f"{pkg_name}{extras}"
            print(f"Installing {pkg_install}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_install])
            return importlib.import_module(import_name)
        except Exception as e:
            print(f"Failed to install or import {pkg_name}: {e}")
            return None

# Install and import XGBoost for GPU support
# Try local conda first for GPU support, fallback to pip
xgb = None
conda_path = os.path.join(os.getcwd(), "miniconda", "Scripts", "conda.exe")
if os.path.exists(conda_path):
    try:
        result = subprocess.run([conda_path, '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("Local conda detected, installing XGBoost with GPU support via conda...")
            # Use conda to install XGBoost with GPU support
            subprocess.check_call([conda_path, 'install', '-c', 'conda-forge', 'xgboost', '-y'])
            # Try to import from conda environment
            # Add conda to PATH temporarily
            conda_bin = os.path.join(os.getcwd(), "miniconda", "Scripts")
            env = os.environ.copy()
            env['PATH'] = conda_bin + os.pathsep + env['PATH']
            # Try importing xgboost
            try:
                import sys
                sys.path.insert(0, os.path.join(os.getcwd(), "miniconda", "Lib", "site-packages"))
                xgb = __import__('xgboost')
                print("XGBoost imported from conda environment")
            except ImportError:
                print("Failed to import XGBoost from conda, falling back to pip")
                xgb = install_and_import('xgboost')
        else:
            raise Exception("Conda not working")
    except Exception as e:
        print(f"Conda install failed ({e}), falling back to pip install...")
        xgb = install_and_import('xgboost')
else:
    print("Local conda not found, falling back to pip install...")
    xgb = install_and_import('xgboost')

if xgb:
    from xgboost import XGBRegressor
else:
    print("XGBoost not available, falling back to CPU models")
    XGBRegressor = None

# Verbose flag for toggling notebook prints
VERBOSE = True

# Downsample flag for memory-aware loading
DOWNSAMPLE = True

d:\AnDB\L\mse\railway-delay\notebooks


ModuleNotFoundError: No module named 'src'

In [ ]:
# GPU Training Setup
# Check if GPU is available for XGBoost
def gpu_available():
    try:
        import xgboost as xgb
        import pandas as pd
        # Try to fit a small model with GPU
        X_test = pd.DataFrame({'a': [1,2,3]})
        y_test = [1,2,3]
        test_reg = xgb.XGBRegressor(tree_method='gpu_hist', n_estimators=1, max_depth=1)
        test_reg.fit(X_test, y_test)
        return True
    except Exception as e:
        print(f"GPU not available: {e}")
        return False

print("GPU available for XGBoost:", gpu_available())

In [ ]:
# Test GPU training with a small sample
if XGBRegressor is not None:
    print("Testing XGBoost training...")
    import pandas as pd
    # Create a small dummy dataset
    X_test = pd.DataFrame({'feature1': [1,2,3,4,5], 'feature2': [5,4,3,2,1]})
    y_test = [1,2,3,4,5]
    
    # Fit a quick model
    tree_method = 'gpu_hist' if gpu_available() else 'hist'
    test_model = XGBRegressor(tree_method=tree_method, n_estimators=10, random_state=RANDOM_STATE)
    test_model.fit(X_test, y_test)
    print(f"XGBoost training test successful with tree_method='{tree_method}'!")
else:
    print("XGBoost not available for testing")

In [ ]:
# ---

## Section 2: Load Data (Train & Test)


# Attempt to read a sample to infer columns & dtypes
sample_nrows = 5000
try:
    train_sample = pd.read_csv(DATATRAIN, nrows=sample_nrows)
    print("Train sample loaded. Shape:", train_sample.shape)
    print(train_sample.columns.tolist()[:50])
except Exception as e:
    print("Failed to read train sample:", e)
    train_sample = None

try:
    test_sample = pd.read_csv(DATATEST, nrows=sample_nrows)
    print("Test sample loaded. Shape:", test_sample.shape)
    print(test_sample.columns.tolist()[:50])
except Exception as e:
    print("Failed to read test sample:", e)
    test_sample = None

# Detect time columns (scheduled/actual)
possible_time_columns = [c for c in (train_sample.columns if train_sample is not None else []) if "DATE" in c.upper() or "DEPART" in c.upper() or "TIME" in c.upper()]

TIME_COLS = {"scheduled": None, "actual": None}
for c in (train_sample.columns if train_sample is not None else []):
    cu = c.upper()
    if "SCHEDULED" in cu and TIME_COLS["scheduled"] is None:
        TIME_COLS["scheduled"] = c
    if "ACTUAL" in cu and TIME_COLS["actual"] is None:
        TIME_COLS["actual"] = c

if TIME_COLS["scheduled"] is None:
    for c in (train_sample.columns if train_sample is not None else []):
        if "DEPARTURE" in c.upper() or "SCHEDULE" in c.upper():
            TIME_COLS["scheduled"] = c
            break

if TIME_COLS["actual"] is None:
    for c in (train_sample.columns if train_sample is not None else []):
        if "ACTUAL" in c.upper() or "ARRIVAL" in c.upper():
            TIME_COLS["actual"] = c
            break

print("Detected time columns:", TIME_COLS)

parse_dates_cols = [col for col in TIME_COLS.values() if col]
load_kwargs = {"parse_dates": parse_dates_cols} if parse_dates_cols else {}

use_nrows = None
if DOWNSAMPLE:
    use_nrows = MAX_ROWS

# Load full dataframes (memory-aware)
try:
    train_df = pd.read_csv(DATATRAIN, nrows=use_nrows, parse_dates=parse_dates_cols)
    print("Train shape:", train_df.shape)
except MemoryError:
    print('MemoryError during full read, falling back to DOWNSAMPLE using MAX_ROWS')
    use_nrows = MAX_ROWS
    train_df = pd.read_csv(DATATRAIN, nrows=use_nrows, parse_dates=parse_dates_cols)
    print("Train shape (downsampled):", train_df.shape)

if os.path.exists(DATATEST):
    try:
        test_df = pd.read_csv(DATATEST, nrows=use_nrows, parse_dates=parse_dates_cols)
        print("Test shape:", test_df.shape)
    except Exception as e:
        print('Test load failed:', e)
        test_df = None
else:
    test_df = None
    print("Test path not found.")

In [ ]:
# ---

## Section 3: Initial Data Inspection & Quick Cleaning

# Basic info and sample head
train_df.info()
train_df.head()

# ID column detection and cast to str
id_cols = [c for c in train_df.columns if ("ID" in c.upper() or c.upper().endswith("_ID"))]
print("ID columns:", id_cols)
for c in id_cols:
    train_df[c] = train_df[c].astype(str)

# Ensure DELAY_MINUTES exists or compute
if "DELAY_MINUTES" not in train_df.columns:
    if TIME_COLS.get("scheduled") and TIME_COLS.get("actual"):
        sched_col = TIME_COLS["scheduled"]
        act_col = TIME_COLS["actual"]
        print(f"Attempting to compute DELAY_MINUTES from {sched_col} and {act_col}...")

        def robust_parse_datetime(series, base_date_col=None):
            """Parse a pandas Series into datetimes handling several common formats.
               Returns a pandas Series of dtype datetime64[ns] with NaT where parsing failed.
            """
            s = series.copy()

            # Primary try: pandas parser
            dt = pd.to_datetime(s, errors='coerce', infer_datetime_format=True)
            success = dt.notna().mean()
            if success > 0.7:
                return dt

            # Heuristic: numeric values that represent hour-of-day as float (e.g., 2.0 -> 02:00)
            if pd.api.types.is_numeric_dtype(s):
                mx = s.max(skipna=True)
                if mx is not None and mx <= 24:
                    # treat values as hour floats
                    base_date = pd.Timestamp("1970-01-01")
                    dt2 = base_date + pd.to_timedelta(s.astype(float), unit='h')
                    return dt2
                if mx is not None and mx <= 2359:
                    # treat as HHMM integer
                    ints = s.fillna(0).astype(int).astype(str).str.zfill(4)
                    dt2 = pd.to_datetime(ints, format='%H%M', errors='coerce')
                    return dt2
                # If very large -> epoch seconds or ms
                if mx is not None and mx > 1e9:
                    # choose ms if likely
                    unit = 's'
                    if mx > 1e12:
                        unit = 'us'
                    elif mx > 1e10:
                        unit = 'ms'
                    dt2 = pd.to_datetime(s, unit=unit, errors='coerce')
                    return dt2

            # If strings like '2.0', try interpret as hour float
            s_str = s.astype(str).str.strip()
            mask_hour = s_str.str.match(r'^\d+\.?\d*$')
            if mask_hour.any():
                try:
                    numeric = s_str[mask_hour].astype(float)
                    base_date = pd.Timestamp("1970-01-01")
                    dt_temp = base_date + pd.to_timedelta(numeric, unit='h')
                    result = pd.Series(pd.NaT, index=s.index, dtype='datetime64[ns]')
                    result.loc[mask_hour] = dt_temp
                    # Also try to fill other entries by parsing directly
                    other_mask = ~mask_hour
                    if other_mask.any():
                        result.loc[other_mask] = pd.to_datetime(s_str[other_mask], errors='coerce', infer_datetime_format=True)
                    return result
                except Exception:
                    pass

            # Try replacing '.' with ':' (e.g., '02.30' -> '02:30')
            s_colon = s_str.str.replace('.', ':', regex=False)
            dt3 = pd.to_datetime(s_colon, errors='coerce', infer_datetime_format=True)
            if dt3.notna().mean() > 0.3:
                return dt3

            # If we have a base_date_col available (like SCHEDULED_DT or DATE), combine date with time-of-day patterns
            if base_date_col is not None and base_date_col in train_df.columns:
                try:
                    base_dates = pd.to_datetime(train_df[base_date_col], errors='coerce')
                    # For numeric or short string times, try create HH:MM
                    # handle floats hours
                    if mask_hour.any():
                        result = base_dates.copy().astype('datetime64[ns]')
                        hours = s_str[mask_hour].astype(float)
                        result.loc[mask_hour] = base_dates.loc[mask_hour] + pd.to_timedelta(hours, unit='h')
                        return result
                except Exception:
                    pass

            # fallback to try to coerce with infer format
            return dt

        sched_dt = robust_parse_datetime(train_df[sched_col], base_date_col='SCHEDULED_DT')
        act_dt = robust_parse_datetime(train_df[act_col], base_date_col='SCHEDULED_DT')

        parsed_info = {
            'scheduled_parsed': float(sched_dt.notna().mean()),
            'actual_parsed': float(act_dt.notna().mean())
        }
        print('Parsing success rates:', parsed_info)

        # If both parsed reasonably well, compute delay
        if (sched_dt.notna().mean() > 0.05) and (act_dt.notna().mean() > 0.05):
            train_df['SCHEDULED_DT'] = sched_dt.where(sched_dt.notna(), train_df.get('SCHEDULED_DT'))
            train_df['ACTUAL_DT'] = act_dt
            train_df['DELAY_MINUTES'] = (train_df['ACTUAL_DT'] - train_df['SCHEDULED_DT']).dt.total_seconds() / 60
            print('Computed DELAY_MINUTES; NaN count:', train_df['DELAY_MINUTES'].isnull().sum())
        else:
            print('Warning: Could not reliably parse scheduled/actual columns into datetimes.\n  - scheduled parsed fraction:', parsed_info['scheduled_parsed'], '\n  - actual parsed fraction:', parsed_info['actual_parsed'])
            # As fallback, if a numeric 'DELAY_MINUTES' column exists in a different column name, try to find it
            # e.g., 'DELAY' or 'DELAY_MIN' or 'DELAY_MINUTES'
            possible_delay_cols = [c for c in train_df.columns if 'DELAY' in c.upper() and 'MIN' in c.upper()]
            if possible_delay_cols:
                print('Found possible delay columns:', possible_delay_cols)
                # Use the first candidate
                train_df['DELAY_MINUTES'] = train_df[possible_delay_cols[0]].astype(float)
            else:
                # set to NaN and warn
                train_df['DELAY_MINUTES'] = np.nan
                print('No fallback delay column found; DELAY_MINUTES set to NaN')

    else:
        raise ValueError("DELAY_MINUTES not present and time columns not detected.")

# Derive time features
if TIME_COLS.get("scheduled"):
    # Ensure SCHEDULED_DT exists; try to parse if not
    if 'SCHEDULED_DT' not in train_df.columns:
        try:
            train_df['SCHEDULED_DT'] = pd.to_datetime(train_df[TIME_COLS['scheduled']], errors='coerce', infer_datetime_format=True)
        except Exception:
            train_df['SCHEDULED_DT'] = pd.NaT

    train_df["HOUR"] = train_df["SCHEDULED_DT"].dt.hour
    train_df["DATE"] = train_df["SCHEDULED_DT"].dt.date
    train_df["MONTH"] = train_df["SCHEDULED_DT"].dt.month
    train_df["WEEKDAY"] = train_df["SCHEDULED_DT"].dt.weekday

# Missingness summary
missing_summary = train_df.isnull().mean().sort_values(ascending=False)
print(missing_summary.head(30))

# Drop duplicates
before_dups = len(train_df)
train_df.drop_duplicates(inplace=True)
print(f"Dropped {before_dups - len(train_df)} exact duplicates")

# Quick view of numeric columns
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns sample:", numeric_cols[:40])

print('Preview:')
train_df.head(3)

In [ ]:
# Debug: show presence of demo variables and last RESULTS
if 'RESULTS' not in globals():
    RESULTS = []
for name in ['rf_pipeline', 'rf_demo', 'pred_demo', 'y_val_demo', 'y_true_demo', 'results_df']:
    print(name, 'in globals():', name in globals())

print('\nLast RESULTS item:')
print(RESULTS[-1] if len(RESULTS) > 0 else 'No results')

In [ ]:
# Fallback to compute HOUR from scheduled column (if SCHEDULED_DT parsing failed)
if TIME_COLS.get('scheduled') and ('HOUR' not in train_df.columns or train_df['HOUR'].isnull().all()):
    sc = TIME_COLS['scheduled']
    if sc in train_df.columns:
        s = train_df[sc]
        if pd.api.types.is_numeric_dtype(s):
            # treat numeric less than 24 as hour
            if s.max(skipna=True) <= 24:
                train_df['HOUR'] = s.fillna(0).astype(float).apply(lambda x: int(np.floor(x)))
        else:
            # try to extract leading hour from string patterns
            s_str = s.astype(str)
            # match HH or HH:MM
            import re
            def extract_hour(ss):
                if pd.isna(ss):
                    return np.nan
                m = re.match(r"(\d{1,2})[:\.\s-]?", ss)
                if m:
                    try:
                        return int(m.group(1))
                    except:
                        return np.nan
                return np.nan
            train_df['HOUR'] = s_str.apply(extract_hour)

# Convert HOUR to integer or NaN
train_df['HOUR'] = train_df['HOUR'].apply(lambda x: int(x) if not pd.isna(x) else np.nan)


In [ ]:
# ---

## Section 4: Target Handling & Missing Value Imputation

# 4.1 Handle negative DELAY_MINUTES values
NEGATIVE_TO_ZERO = True
if NEGATIVE_TO_ZERO:
    n_neg = (train_df['DELAY_MINUTES'] < 0).sum()
    print(f"Negative delays: {n_neg} rows. Setting them to 0.")
    train_df.loc[train_df['DELAY_MINUTES'] < 0, 'DELAY_MINUTES'] = 0

# 4.2 Missing value imputation: numeric groupwise median and categorical constant
# Numeric columns - do groupwise median imputation on station and hour for known fields
numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

# Identify numeric features to impute (exclude target and date columns)
exclude_cols = ['DELAY_MINUTES'] + ["SCHEDULED_DT"]
impute_numeric_cols = [c for c in numeric_cols if c not in exclude_cols]

# Example groupby median for a subset
for c in impute_numeric_cols:
    if train_df[c].isnull().sum() > 0:
        try:
            gp_med = train_df.groupby(["STATION_ID", "HOUR"])[c].transform("median")
            train_df[c] = train_df[c].fillna(gp_med)
            # fallback to global median
            train_df[c] = train_df[c].fillna(train_df[c].median())
        except Exception:
            train_df[c] = train_df[c].fillna(train_df[c].median())

# Fill categorical missing with 'Unknown'
cat_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
for c in cat_cols:
    train_df[c] = train_df[c].fillna('Unknown')

# Confirm missing values
print(train_df.isnull().sum().loc[lambda x: x>0])


In [ ]:
# ---

## Section 5: Outlier Detection & Winsorization

# Compute quantiles
q01 = train_df['DELAY_MINUTES'].quantile(0.01)
q99 = train_df['DELAY_MINUTES'].quantile(0.99)
mean_val = train_df['DELAY_MINUTES'].mean()
print(f"Delay quantiles: 1%={q01:.2f}, 99%={q99:.2f}, mean={mean_val:.2f}")

# Use explicit WINSOR_CAP defined in setup but fallback to q99 if smaller
winsor_cap = min(q99, WINSOR_CAP) if 'WINSOR_CAP' in globals() else q99
print(f"Winsorization cap (using min(q99, WINSOR_CAP)): {winsor_cap}")

# ... rest of plotting code unchanged ...
plt.figure(figsize=(12,5))
sns.histplot(train_df['DELAY_MINUTES'], bins=80, kde=False)
plt.title('DELAY_MINUTES distribution')
plt.xlabel('Minutes')
plt.xlim(0, min(train_df['DELAY_MINUTES'].quantile(0.999), 500))
plt.show()

plt.figure(figsize=(6,3))
sns.boxplot(x=train_df['DELAY_MINUTES'])
plt.xlim(0, min(train_df['DELAY_MINUTES'].quantile(0.999), 500))
plt.title('Boxplot of DELAY_MINUTES')
plt.show()

# Winsorization at the chosen cap
train_df['DELAY_MINUTES_WINSORIZED'] = np.minimum(train_df['DELAY_MINUTES'], winsor_cap)
rows_clipped = (train_df['DELAY_MINUTES'] > winsor_cap).sum()
print(f"Winsorization cap: {winsor_cap}. Rows clipped: {rows_clipped}")

# For further steps we'll use the winsorized target
train_df['TARGET'] = train_df['DELAY_MINUTES_WINSORIZED']

In [ ]:
# ---

## Section 6: Target Transformation (Log1p) & Inverse Transform

train_df['TARGET_LOG1P'] = np.log1p(train_df['TARGET'])

# Transform functions
inv_log1p = lambda x: np.expm1(x)

print('Original target skew:', train_df['TARGET'].skew())
print('Transformed (log1p) target skew:', train_df['TARGET_LOG1P'].skew())

# Plot distribution after transformation
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(train_df['TARGET'], bins=80)
plt.title('Target (winsorized) distribution')
plt.subplot(1,2,2)
sns.histplot(train_df['TARGET_LOG1P'], bins=80)
plt.title('Target log1p distribution')
plt.show()


In [ ]:
# ---

## Section 7: EDA - Univariate & Multivariate

# Basic descriptive statistics
print('Target (winsorized) stats:')
print(train_df['TARGET'].describe())

# Show skewness/kurtosis
from scipy.stats import skew, kurtosis
print('Skew:', skew(train_df['TARGET']))
print('Kurtosis:', kurtosis(train_df['TARGET']))

# Average delay by HOUR
hour_grp = train_df.groupby('HOUR')['TARGET'].mean().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot(x='HOUR', y='TARGET', data=hour_grp)
plt.title('Average delay by hour of day')
plt.show()

# Average delay by month
month_grp = train_df.groupby('MONTH')['TARGET'].mean().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot(x='MONTH', y='TARGET', data=month_grp)
plt.title('Average delay by month')
plt.show()

# Top 10 busiest stations by count
if 'STATION_ID' in train_df.columns:
    busy_stations = train_df['STATION_ID'].value_counts().nlargest(10).index
    plt.figure(figsize=(12,6))
    sns.boxplot(x='STATION_ID', y='TARGET', data=train_df[train_df['STATION_ID'].isin(busy_stations)])
    plt.title('Delay distribution for top 10 stations')
    plt.xticks(rotation=45)
    plt.show()

# Correlation matrix among selected numeric columns
num_cols_for_corr = ['TARGET'] + [c for c in numeric_cols if c not in ['DELAY_MINUTES','TARGET','TARGET_LOG1P']][:8]
plt.figure(figsize=(8,6))
sns.heatmap(train_df[num_cols_for_corr].corr(), annot=True, fmt='.2f')
plt.title('Correlation matrix')
plt.show()


In [ ]:
# ---

## Section 8: Feature Engineering: Time, Lag, Rolling, Interactions

# Time cyclical features
train_df['HOUR_SIN'] = np.sin(2 * np.pi * train_df['HOUR'] / 24)
train_df['HOUR_COS'] = np.cos(2 * np.pi * train_df['HOUR'] / 24)

# Compute lag and rolling features using the helper functions (safe fallback if TRAIN_ID/STATION_ID missing)
train_df = compute_prev_delay_safe(train_df)
train_df = compute_rolling_features_safe(train_df)


In [ ]:
# ---

# Quick small test for compute_prev_delay and compute_rolling_features - sanity check
import pandas as pd, numpy as np
# with route column
df_test = pd.DataFrame({ 'SCHEDULED_DT': pd.date_range('2020-01-01', periods=5, freq='D'), 'TARGET':[10,20,5,0,3], 'STATION_ID':['A','A','A','B','B'] })
# without route column (simulate dataset with no TRAIN_ID/STATION_ID)
df_test_no_route = df_test.drop(columns=['STATION_ID']).copy()
print('route present:', _get_route_column(df_test))
print('route missing:', _get_route_column(df_test_no_route))
print('Compute prev delay with route:')
print(compute_prev_delay(df_test))
print('Compute prev delay without route:')
print(compute_prev_delay(df_test_no_route))
print('Compute rolling with route:')
print(compute_rolling_features(df_test))
print('Compute rolling without route:')
print(compute_rolling_features(df_test_no_route))

In [ ]:
# ---

## Section 9: Feature Encoding & Scaling with Pipelines

# Select features automatically
candidate_features = []

# Numeric features
numeric_features = ['DISTANCE', 'STOPS', 'PASSENGER_LOAD', 'PREV_DELAY', 'ROLLING_MEAN_DELAY_7D', 'WEATHER_IMPACT', 'HOUR_SIN', 'HOUR_COS']
numeric_features = [f for f in numeric_features if f in train_df.columns]

# Categorical features (IDs and operators)
cat_features_all = id_cols + [c for c in ['OPERATOR_ID', 'ROUTE_ID', 'TRAIN_TYPE', 'STATION_NAME'] if c in train_df.columns]
cat_features_all = [c for c in cat_features_all if c in train_df.columns]

# Auto selection small cardinality for one-hot encoding
onehot_features, label_features = [], []
for c in cat_features_all:
    nunique = train_df[c].nunique()
    if nunique <= 50:
        onehot_features.append(c)
    else:
        label_features.append(c)

print('Numeric features:', numeric_features)
print('OneHot features:', onehot_features)
print('Label/High-card features:', label_features)

# For label features we'll use simple ordinal encoding using scikit-learn's OrdinalEncoder or mapping.
from sklearn.preprocessing import OrdinalEncoder

if label_features:
    # Fit mapping using fitted categories
    oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    train_df[label_features] = oe.fit_transform(train_df[label_features].fillna('Unknown'))

# Build pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, onehot_features)
    ],
    remainder='passthrough'  # keep other features if needed
)

# Final small helper to assemble X and y
FEATURES = numeric_features + onehot_features + label_features
print('Final FEATURES list count:', len(FEATURES))

X = train_df[FEATURES]
y = train_df['TARGET_LOG1P']  # model on the log scale

# A quick check of shape
print('X shape:', X.shape, 'y shape:', y.shape)


In [ ]:
# ---

## Section 10: Prepare Train/Validation Splits (Time Series)

# Use a time cutoff: last n days as holdout
holdout_days = 60
max_date = train_df['SCHEDULED_DT'].max()
train_cutoff = max_date - pd.Timedelta(days=holdout_days)

train_idx = train_df['SCHEDULED_DT'] < train_cutoff
val_idx = train_df['SCHEDULED_DT'] >= train_cutoff

X_train = X.loc[train_idx]
X_val = X.loc[val_idx]
y_train = y.loc[train_idx]
y_val = y.loc[val_idx]

print('Train set:', X_train.shape, 'Validation set:', X_val.shape)

# For cross-validation we'll build a TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=5)
print('TimeSeriesSplit created with 5 splits')


In [ ]:
# ---

## Section 11: Baseline Models (Mean & Linear Regression)

# Baseline: predict mean of y_train (in log space)
mean_train_log = y_train.mean()

# Predictions (log space):
baseline_pred_log = np.full(len(y_val), mean_train_log)
baseline_pred = np.expm1(baseline_pred_log)

y_val_original = np.expm1(y_val)

metrics_baseline = metrics_summary(y_val_original, baseline_pred)
metrics_baseline

# Baseline 2: Linear Regression
linear_pipe = Pipeline(steps=[('preproc', preprocessor), ('linear', LinearRegression())])
linear_pipe.fit(X_train, y_train)

# Predict and inverse transform
y_pred_log_lin = linear_pipe.predict(X_val)
y_pred_lin = np.expm1(y_pred_log_lin)

metrics_linear = metrics_summary(y_val_original, y_pred_lin)

print('Baseline metrics (original scale):', metrics_baseline)
print('Linear Regression metrics (original scale):', metrics_linear)

RESULTS.append({'model':'Baseline Mean', **metrics_baseline})
RESULTS.append({'model':'Linear Regression', **metrics_linear})


In [ ]:
# ---

## Section 12: Random Forest Regressor: Train & Evaluate

rf_pipe = Pipeline(steps=[('preproc', preprocessor), ('rf', RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))])

# Hyperparameter space
rf_param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [10, 20, None],
    'rf__min_samples_leaf': [1, 3, 5],
    'rf__max_features': ['auto', 'sqrt']
}

# Time-aware CV via TimeSeriesSplit
tscv_rf = TimeSeriesSplit(n_splits=3)

rf_search = RandomizedSearchCV(rf_pipe, rf_param_grid, n_iter=6, cv=tscv_rf, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=1)
start=time.time()
rf_search.fit(X_train, y_train)
end=time.time()

print('Best RF params:', rf_search.best_params_)
print('Best RF CV score (neg RMSE in log space):', rf_search.best_score_)

# Evaluate on validation set
rf_best = rf_search.best_estimator_
rf_pred_log = rf_best.predict(X_val)
rf_pred = np.expm1(rf_pred_log)
metrics_rf = metrics_summary(y_val_original, rf_pred)
print('Random Forest metrics (original scale):', metrics_rf)

RESULTS.append({'model':'Random Forest', **metrics_rf, 'tuning_time_s': end-start})

# Feature importances (approx) - use numeric feature mapping
try:
    fi = rf_best.named_steps['rf'].feature_importances_
    # Handle columns after preprocessing for the number of features
    # We can compute feature names from preprocessor
    # This is non-trivial with onehot; we provide the top numeric feature importance mapping approximately
    print('RandomForest feature importances (sample):', fi[:10])
except Exception as e:
    print('Failed to extract RF feature importances:', e)


In [ ]:
# ---

## Section 13: XGBoost / LightGBM Champion Model with Hyperparameter Tuning

if xgb is None and lgb is None:
    print('Both XGBoost and LightGBM are not available. Skipping gradient boosting training.')
else:
    if xgb is not None:
        print('Training XGBoost as the champion model (if available)')
        xgb_pipe = Pipeline(steps=[('preproc', preprocessor), ('xgb', xgb.XGBRegressor(objective='reg:squarederror', random_state=RANDOM_STATE, n_jobs=-1, verbosity=1))])

        xgb_param_grid = {
            'xgb__n_estimators': [100, 300, 500],
            'xgb__learning_rate': [0.01, 0.05, 0.1],
            'xgb__max_depth': [4, 6, 8],
            'xgb__reg_alpha': [0, 0.1, 1],
            'xgb__reg_lambda': [1, 10],
            'xgb__min_child_weight': [1, 5, 10],
        }

        search_iter = 12
        xgb_search = RandomizedSearchCV(xgb_pipe, xgb_param_grid, n_iter=search_iter, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=2)

        start = time.time()
        xgb_search.fit(X_train, y_train)
        end = time.time()

        print('Best XGB params', xgb_search.best_params_)
        print('Best XGB CV score (neg RMSE log-scale):', xgb_search.best_score_)

        # Evaluate on validation set
        xgb_best = xgb_search.best_estimator_
        xgb_pred_log = xgb_best.predict(X_val)
        xgb_pred = np.expm1(xgb_pred_log)
        metrics_xgb = metrics_summary(y_val_original, xgb_pred)
        print('XGBoost metrics (original scale):', metrics_xgb)

        RESULTS.append({'model':'XGBoost (tuned)', **metrics_xgb, 'tuning_time_s': end-start})
    else:
        # If XGBoost not available, use LightGBM if present
        if lgb is not None:
            print('XGBoost not found, using LightGBM (as fallback).')
            lgb_pipe = Pipeline(steps=[('preproc', preprocessor), ('lgb', lgb.LGBMRegressor(random_state=RANDOM_STATE, n_jobs=-1))])

            lgb_param_grid = {
                'lgb__n_estimators': [100, 300, 500],
                'lgb__learning_rate': [0.01, 0.05, 0.1],
                'lgb__max_depth': [-1, 10, 20],
                'lgb__num_leaves': [31, 50, 100],
                'lgb__min_child_samples': [5, 20],
            }

            lgb_search = RandomizedSearchCV(lgb_pipe, lgb_param_grid, n_iter=12, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=RANDOM_STATE, verbose=2)
            start = time.time()
            lgb_search.fit(X_train, y_train)
            end = time.time()

            print('Best LGB params', lgb_search.best_params_)
            print('Best LGB CV score (neg RMSE log-scale):', lgb_search.best_score_)

            lgb_best = lgb_search.best_estimator_
            lgb_pred_log = lgb_best.predict(X_val)
            lgb_pred = np.expm1(lgb_pred_log)
            metrics_lgb = metrics_summary(y_val_original, lgb_pred)
            print('LightGBM metrics (original scale):', metrics_lgb)
            RESULTS.append({'model':'LightGBM (tuned)', **metrics_lgb, 'tuning_time_s': end-start})
        else:
            print('No gradient boosting model available (xgboost/lightgbm)')

In [ ]:
# ---

## Section 14: Model Comparison Table & Evaluation Metrics

results_df = pd.DataFrame(RESULTS)
# round metrics for clarity
for m in ['rmse','mae','r2']:
    for col in results_df.columns:
        if col.endswith(m):
            results_df[col] = results_df[col].round(3)

results_df

# Plot comparison of RMSE
plt.figure(figsize=(8,4))
if 'rmse' in results_df.columns:
    sns.barplot(data=results_df, x='model', y='rmse')
    plt.xticks(rotation=45)
    plt.title('RMSE (lower is better) comparison')
    plt.show()


In [ ]:
# ---

## Section 14b: Model Comparison Summary & Recommendation

# Ensure results_df exists (it is created earlier from RESULTS)
try:
    if 'results_df' not in globals():
        results_df = pd.DataFrame(RESULTS)

    # If RMSE columns are on log or original scale, use 'rmse' (original scale) shown in the table
    if 'rmse' not in results_df.columns and 'rmse' in [c.lower() for c in results_df.columns]:
        # already normalized
        pass

    if 'rmse' in results_df.columns:
        best_row = results_df.loc[results_df['rmse'].idxmin()]
        print('Best model by RMSE:', best_row['model'], 'RMSE:', best_row['rmse'])
        print('\nFull Results:')
        display(results_df.sort_values('rmse'))
    else:
        print('Results dataframe does not include RMSE column; printing RESULTS instead')
        print(RESULTS)

    # Recommendation message based on best
def recommend_model(results_df):
    if 'rmse' in results_df.columns:
        best_idx = results_df['rmse'].idxmin()
        best = results_df.loc[best_idx]
        return f"Recommended model: {best['model']} with RMSE={best['rmse']:.3f}. Consider LightGBM or XGBoost, tuned with Optuna if available."
    return 'No recommendation (no RMSE available)'

print(recommend_model(results_df))

# Add a follow-up suggestion to test ensemble or stacking if single model performance saturates
print('\nSuggestion: If Best model is similar to others, try a lightweight stacking ensemble (avg/stack) or additional features: lag windows, weather, special event flags.')

In [ ]:
# ---

## Section 15b: Top Features & Worst Predictions Summary

# Build a utility to extract feature names from preprocessor (sklearn>=1.0) safely

def get_feature_names(preprocessor, numeric_features, onehot_features, label_features):
    try:
        # ColumnTransformer with get_feature_names_out (sklearn >=1.0)
        names = preprocessor.get_feature_names_out()
        return list(names)
    except Exception:
        # fallback: reconstruct approximate feature names
        feat_names = []
        feat_names.extend(numeric_features)
        # For onehot, try access fitted categories
        try:
            if onehot_features and hasattr(preprocessor.named_transformers_['cat'].named_steps['onehot'], 'categories_'):
                ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
                for i, col in enumerate(onehot_features):
                    cats = ohe.categories_[i]
                    names_cat = [f"{col}_{c}" for c in cats]
                    feat_names.extend(names_cat)
        except Exception:
            feat_names.extend(onehot_features)
        feat_names.extend(label_features)
        return feat_names

# Find a pipeline to extract feature importances: prefer trained best models, else fall back to demo or rf pipeline
best_est = None
for varname in ['xgb_best', 'lgb_best', 'rf_best', 'xgb_opt_pipe', 'xgb_opt', 'rf_pipeline', 'rf_demo']:
    if varname in globals():
        best_est = globals()[varname]
        print('Using model variable:', varname)
        break

if best_est is not None:
    try:
        # extract core model
        core_model = None
        if hasattr(best_est, 'named_steps'):
            core_model = next((v for k, v in best_est.named_steps.items() if hasattr(v, 'feature_importances_')), None)
        else:
            core_model = best_est

        if core_model is not None and hasattr(core_model, 'feature_importances_'):
            fi = core_model.feature_importances_
            feat_names = get_feature_names(preprocessor, numeric_features, onehot_features, label_features)
            if len(feat_names) != len(fi):
                # fallback: map to numeric features only
                feat_names = [f for f in (numeric_features + onehot_features + label_features)][:len(fi)]
            importances = pd.Series(fi, index=feat_names)
            top10 = importances.sort_values(ascending=False).head(10)
            print('\nTop 10 features by importance:')
            display(top10)
        else:
            print('Feature importances not found for core model; attempting permutation importance fallback')
            from sklearn.inspection import permutation_importance
            # Use a small sample to speed up
            if 'X_train' in globals() and len(X_train) > 0:
                sample = X_train.sample(min(200, len(X_train)), random_state=RANDOM_STATE)
                y_sample = y_train.loc[sample.index] if 'y_train' in globals() else None
                r = permutation_importance(best_est, sample, y_sample, scoring='neg_root_mean_squared_error', n_repeats=5, random_state=RANDOM_STATE, n_jobs=1)
                perm_importances = pd.Series(r.importances_mean, index=(get_feature_names(preprocessor, numeric_features, onehot_features, label_features)[:len(r.importances_mean)]))
                top10 = perm_importances.sort_values(ascending=False).head(10)
                print('\nTop 10 features by permutation importance:')
                display(top10)
            else:
                print('No X_train/y_train found; cannot compute permutation importance')
    except Exception as e:
        print('Failed to compute feature importances:', e)
else:
    print('No trained model variable found in globals to compute feature importance.')

# Worst predictions summary
# Try to use 'top50' variable if present; else compute using available val set and a model
try:
    if 'top50' in globals():
        print('\nTop 10 worst predictions (subset of top50):')
        display(top50.head(10)[['TRAIN_ID' if 'TRAIN_ID' in top50.columns else id_cols[0], 'SCHEDULED_DT', 'y_true', 'y_pred', 'abs_error']])
    elif 'val_demo' in globals() and 'pred_demo' in globals():
        val_df_demo = val_demo.copy()
        val_df_demo['y_true'] = y_true_demo
        val_df_demo['y_pred'] = pred_demo
        val_df_demo['abs_error'] = (val_df_demo['y_true'] - val_df_demo['y_pred']).abs()
        top10demo = val_df_demo.nlargest(10, 'abs_error')
        print('\nTop 10 worst predictions (demo):')
        display(top10demo[[c for c in ['TRAIN_NUMBER', 'COACH_ID', 'SCHEDULED_DT', 'y_true', 'y_pred', 'abs_error'] if c in top10demo.columns]])
    else:
        print('\nNo worst predictions available yet in globals')
except Exception as e:
    print('Failed to display worst predictions:', e)

In [ ]:
# ---

## Section 15b: Top Features & Worst Predictions Summary

# Build a utility to extract feature names from preprocessor (sklearn>=1.0) safely

def get_feature_names(preprocessor, numeric_features, onehot_features, label_features):
    try:
        # ColumnTransformer with get_feature_names_out
        names = preprocessor.get_feature_names_out()
        return list(names)
    except Exception:
        # fallback: reconstruct approximate feature names
        feat_names = []
        feat_names.extend(numeric_features)
        # For onehot, try access fitted categories
        try:
            if onehot_features and hasattr(preprocessor.named_transformers_['cat'].named_steps['onehot'], 'categories_'):
                ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
                for i, col in enumerate(onehot_features):
                    cats = ohe.categories_[i]
                    names_cat = [f"{col}_{c}" for c in cats]
                    feat_names.extend(names_cat)
        except Exception:
            feat_names.extend(onehot_features)
        feat_names.extend(label_features)
        return feat_names

# Determine best model and feature importances
best_est = None
if 'xgb_best' in globals():
    best_est = xgb_best
elif 'lgb_best' in globals():
    best_est = lgb_best
elif 'rf_best' in globals():
    best_est = rf_best

if best_est is not None:
    print('Best model found for feature importance extraction:', best_est)
    try:
        # get core model and importances
        core_model = None
        if hasattr(best_est, 'named_steps'):
            core_model = next((v for k, v in best_est.named_steps.items() if hasattr(v, 'feature_importances_')), None)
        else:
            core_model = best_est

        if core_model is not None and hasattr(core_model, 'feature_importances_'):
            fi = core_model.feature_importances_
            feat_names = get_feature_names(preprocessor, numeric_features, onehot_features, label_features)
            if len(feat_names) != len(fi):
                # fallback to numeric features only
                feat_names = numeric_features + onehot_features + label_features
            importances = pd.Series(fi, index=feat_names)
            top10 = importances.sort_values(ascending=False).head(10)
            print('\nTop 10 features by importance:')
            display(top10)
        else:
            print('Feature importances not found for core model; attempting permutation importance as fallback')
            from sklearn.inspection import permutation_importance
            # Use a small sample due to time
            sample = X_train.sample(min(200, len(X_train)), random_state=RANDOM_STATE)
            y_sample = y_train.loc[sample.index]
            r = permutation_importance(best_est, sample, y_sample, scoring='neg_root_mean_squared_error', n_repeats=10, random_state=RANDOM_STATE, n_jobs=1)
            perm_importances = pd.Series(r.importances_mean, index=get_feature_names(preprocessor, numeric_features, onehot_features, label_features)[:len(r.importances_mean)])
            top10 = perm_importances.sort_values(ascending=False).head(10)
            print('\nTop 10 features by permutation importance:')
            display(top10)
    except Exception as e:
        print('Failed to compute feature importances:', e)
else:
    print('No best model found in notebook variables (xgb_best/lgb_best/rf_best). Skipping feature importance.')

# Worst predictions summary already computed earlier as 'top50' in Section 15; show top 10
try:
    if 'top50' in globals():
        print('\nTop 10 worst predictions (subset of top50):')
        display(top50.head(10)[['TRAIN_ID' if 'TRAIN_ID' in top50.columns else id_cols[0], 'SCHEDULED_DT', 'y_true', 'y_pred', 'abs_error']])
    else:
        print('top50 not found — run validation/prediction first to populate worst predictions.')
except Exception as e:
    print('Failed to display worst predictions:', e)

In [ ]:
# ---

## Section 16: Save Trained Model & Inference on DATATEST

# Save preprocessor and model (safe guard to ensure best_model exists).
if 'best_model' in globals() and best_model is not None:
    best_model_name = 'xgboost_tuned' if 'xgb_best' in globals() else ('rf' if 'rf_best' in globals() else 'linear')
    model_path = os.path.join(MODEL_DIR, f"{best_model_name}_model.joblib")
    try:
        joblib.dump(best_model, model_path)
        print('Saved model to', model_path)
    except Exception as e:
        print('Failed to save best_model here:', e)
else:
    print('No best_model is defined yet; the later pipeline-saving cell will set and save the chosen model.')

# Inference helper
def predict_on_test(test_path):
    print('Loading test data...')
    test_df_local = pd.read_csv(test_path, parse_dates=parse_dates_cols)
    # Add derived features used in training
    if TIME_COLS.get('scheduled') in test_df_local.columns:
        test_df_local['SCHEDULED_DT'] = pd.to_datetime(test_df_local[TIME_COLS['scheduled']])
        test_df_local['HOUR'] = test_df_local['SCHEDULED_DT'].dt.hour
        test_df_local['HOUR_SIN'] = np.sin(2 * np.pi * test_df_local['HOUR']/24)
        test_df_local['HOUR_COS'] = np.cos(2 * np.pi * test_df_local['HOUR']/24)

    # Preprocess label features mapping if used
    if label_features:
        try:
            test_df_local[label_features] = oe.transform(test_df_local[label_features].fillna('Unknown'))
        except Exception:
            test_df_local[label_features] = test_df_local[label_features].fillna(-1)

    # PREV_DELAY is not directly available for unseen trips; we set default
    test_df_local['PREV_DELAY'] = test_df_local.get('PREV_DELAY', -1)
    test_df_local['ROLLING_MEAN_DELAY_7D'] = test_df_local.get('ROLLING_MEAN_DELAY_7D', train_df['TARGET'].median())
    test_df_local['WEATHER_IMPACT'] = test_df_local.get('WEATHER_IMPACT', 0.0)

    # Extract features
    X_test_local = test_df_local[FEATURES]

    # Predict (model returns in log domain), inverse transform
    pred_log_test = best_model.predict(X_test_local)
    pred_test = np.expm1(pred_log_test)

    # Save predictions
    out = test_df_local[[TIME_COLS['scheduled']] if TIME_COLS['scheduled'] in test_df_local.columns else ['SCHEDULED_DT']].copy()
    out['PRED_DELAY_MINUTES'] = pred_test
    out['PRED_DELAY_MINUTES_LOG'] = pred_log_test
    out.to_csv(os.path.join(MODEL_DIR, 'test_predictions.csv'), index=False)
    print('Saved predictions to', os.path.join(MODEL_DIR, 'test_predictions.csv'))

# Optionally run on DATATEST
if os.path.exists(DATATEST):
    predict_on_test(DATATEST)
else:
    print('DATATEST not found; skipping inference')

In [ ]:
# ---

## Section 17: Utilities: Logging, Versioning and Reproducibility

# Log package versions
import platform

package_versions = {
    'python_version': platform.python_version(),
    'pandas': pd.__version__,
    'numpy': np.__version__,
    'sklearn': pkg_resources.get_distribution('scikit-learn').version,
}
print(package_versions)

# Save a small experiment info file
exp_info = {
    'date': datetime.now().isoformat(),
    'random_state': RANDOM_STATE,
    'winsor_cap': float(winsor_cap),
    'dowsample': DOWNSAMPLE,
    'best_model': best_model_name,
}
import json
with open(os.path.join(MODEL_DIR, 'experiment_info.json'), 'w') as f:
    json.dump(exp_info, f, indent=2)

print('Saved experiment_info.json in', MODEL_DIR)


# Final suggestions:
# - Run hyperparameter tuning for longer and with Optuna for better model performance
# - Add feature importance & SHAP analysis for interpretability
# - Optionally use GPU and distributed training for scale

print('Notebook complete. Next steps: cross-check columns, run full tuning, analyze worst cases and create monitoring/alerting.')

## Optional Enhancements (Optuna & SHAP)

Below are optional steps to improve model quality and interpretability:
1. Optuna hyperparameter tuning (XGBoost/LightGBM).  
2. SHAP summary and dependence plots for the final model.  
3. Store metrics and parameters in `models/` for experiment tracking.

In [ ]:
# ---
# Optional: Optuna-based hyperparameter tuning for XGBoost (faster and more flexible than RandomizedSearch)

# We installed optuna and xgboost earlier via the setup helper; check availability

from sklearn.model_selection import cross_val_score

if optuna is not None and xgb is not None and 'X_train' in globals() and 'y_train' in globals():
    print('Running Optuna tuning for XGBoost (optional). This may take a while and uses TimeSeriesSplit CV.')

    def optuna_optimize_xgb(trial):
        param = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'booster': 'gbtree',
            'tree_method': 'gpu_hist' if gpu_available() else 'hist',
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
            'n_estimators': int(trial.suggest_categorical('n_estimators', [100,200,300,500])),
            'random_state': RANDOM_STATE,
        }

        xgb_local = xgb.XGBRegressor(**param, verbosity=0, n_jobs=-1)
        pipe_local = Pipeline(steps=[('preproc', preprocessor), ('xgb', xgb_local)])
        try:
            scores = cross_val_score(pipe_local, X_train, y_train, scoring='neg_root_mean_squared_error', cv=tscv, n_jobs=1)
            return -1.0 * scores.mean()
        except Exception as e:
            print('Optuna trial failed on CV:', e)
            return float('inf')

    study = optuna.create_study(direction='minimize')
    study.optimize(optuna_optimize_xgb, n_trials=20, show_progress_bar=True)

    print('Optuna best value (RMSE):', study.best_value)
    print('Optuna best params:', study.best_params)

    # Fit best estimator on the full training set and evaluate
    best_params = study.best_params
    if 'n_estimators' in best_params:
        best_params['n_estimators'] = int(best_params['n_estimators'])
    best_params['random_state'] = RANDOM_STATE
    best_params['tree_method'] = 'gpu_hist' if gpu_available() else 'hist'
    xgb_opt = xgb.XGBRegressor(**best_params, verbosity=1, n_jobs=-1)

    xgb_opt_pipe = Pipeline(steps=[('preproc', preprocessor), ('xgb', xgb_opt)])
    xgb_opt_pipe.fit(X_train, y_train)

    pred_log_opt = xgb_opt_pipe.predict(X_val)
    pred_opt = np.expm1(pred_log_opt)
    metrics_opt = metrics_summary(y_val_original, pred_opt)
    print('Optuna XGBoost metrics:', metrics_opt)
    RESULTS.append({'model': 'XGBoost (Optuna)', **metrics_opt})
else:
    print('Optuna or XGBoost not available (skipping Optuna tuning).')

In [ ]:
# ---
# Optional: SHAP explainability for the final/best model

try:
    import shap
except Exception:
    import sys
    try:
        !{sys.executable} -m pip install shap
        import shap
    except Exception:
        shap = None
        print('SHAP not installed; skipping SHAP analysis.')

# Choose the explainer model: prefer the optuna-fitted one, then the best pipelined model found earlier
explainer_model = None
if 'xgb_opt_pipe' in globals():
    explainer_model = xgb_opt_pipe
elif 'xgb_best' in globals():
    explainer_model = xgb_best
elif 'rf_best' in globals():
    explainer_model = rf_best
else:
    explainer_model = linear_pipe

if shap is not None and explainer_model is not None and 'X_train' in globals():
    try:
        sample_size = min(500, X_train.shape[0])
        X_shap = X_train.sample(sample_size, random_state=RANDOM_STATE)

        # If pipeline has preprocessor, transform the sample X
        if hasattr(explainer_model, 'named_steps') and 'preproc' in explainer_model.named_steps:
            X_shap_trans = explainer_model.named_steps['preproc'].transform(X_shap)
        else:
            X_shap_trans = X_shap.values

        # get core model
        if hasattr(explainer_model, 'named_steps') and ('xgb' in explainer_model.named_steps or 'rf' in explainer_model.named_steps):
            core_model = explainer_model.named_steps.get('xgb') or explainer_model.named_steps.get('rf')
            explainer = shap.TreeExplainer(core_model)
        else:
            # fallback to an explanation method less suited for linear/other models
            core_model = explainer_model.named_steps.get('linear') if hasattr(explainer_model, 'named_steps') else explainer_model
            explainer = shap.KernelExplainer(core_model.predict, X_shap_trans[:50])

        shap_values = explainer.shap_values(X_shap_trans)

        # Summary plot
        plt.figure(figsize=(10,6))
        shap.summary_plot(shap_values, X_shap_trans, show=False)
        shap_summary_path = os.path.join(MODEL_DIR, 'shap_summary.png')
        plt.tight_layout()
        plt.savefig(shap_summary_path, dpi=150)
        plt.close()
        print('Saved SHAP summary to', shap_summary_path)

        # Save an interactive HTML force plot for the first sample
        try:
            shap_html = os.path.join(MODEL_DIR, 'shap_force_plot.html')
            shap.initjs()
            # create a force plot for the mean of the sample
            f = shap.force_plot(explainer.expected_value, shap_values[0,:], X_shap_trans[0,:], matplotlib=False)
            shap.save_html(shap_html, f)
            print('Saved SHAP force plot to', shap_html)
        except Exception as e:
            print('Could not produce interactive SHAP save:', e)

    except Exception as e:
        print('SHAP analysis failed:', e)
else:
    print('SHAP analysis skipped (shap not installed or no model/data available)')

In [ ]:
# ---
# Save results/metrics to CSV for experiment tracking and model params

if len(RESULTS) > 0:
    try:
        metrics_out = pd.DataFrame(RESULTS)
        metrics_out_file = os.path.join(MODEL_DIR, 'metrics_summary.csv')
        metrics_out.to_csv(metrics_out_file, index=False)
        print('Saved metrics summary to', metrics_out_file)
    except Exception as e:
        print('Failed to save metrics summary:', e)

# Append experiment info and metrics to a log file with timestamp
try:
    import datetime, json
    exp_log = os.path.join(MODEL_DIR, 'metrics_log.csv')
    exp_entry = globals().get('exp_info', {})
    exp_entry.update({'metrics': RESULTS})

    if not os.path.exists(exp_log):
        with open(exp_log, 'w', newline='') as f:
            f.write('timestamp,entry\n')
    with open(exp_log, 'a', newline='') as f:
        f.write(datetime.datetime.now().isoformat() + ',' + json.dumps(exp_entry).replace('\n', '') + '\n')
    print('Appended experiment entry to', exp_log)
except Exception as e:
    print('Failed to append experiment log:', e)

# Save model params if a best model exists
try:
    model_params_file = os.path.join(MODEL_DIR, 'model_params.json')
    if 'best_model' in globals() and hasattr(best_model, 'get_params'):
        params = best_model.get_params()
    elif 'xgb_opt_pipe' in globals():
        # prefer optuna result
        params = xgb_opt_pipe.named_steps['xgb'].get_params()
    elif 'xgb_best' in globals():
        params = xgb_best.named_steps['xgb'].get_params()
    elif 'rf_best' in globals():
        params = rf_best.named_steps['rf'].get_params()
    else:
        params = {'model': 'none'}

    with open(model_params_file, 'w') as f:
        json.dump(params, f, indent=2)
    print('Saved model params JSON to', model_params_file)
except Exception as e:
    print('Could not log model params:', e)


In [ ]:
# Quick check: print results summary (if created by demo)
try:
    if 'results_df' in globals():
        print('results_df (head):')
        display(results_df.head())
    else:
        print('RESULTS list:')
        display(pd.DataFrame(RESULTS))
except Exception as e:
    print('Could not display results:', e)

## Feature Dictionary

- `TRAIN_ID`: Unique identifier for a train (string)
- `STATION_ID`: Station identifier (string)
- `SCHEDULED_DT`: Scheduled datetime for departure/arrival (datetime)
- `ACTUAL_DT`: Actual datetime for departure/arrival (datetime)
- `DELAY_MINUTES`: Observed delay in minutes (float)
- `TARGET`: Winsorized target used for modelling (float)
- `TARGET_LOG1P`: Log1p transformed TARGET for training
- `PREV_DELAY`: Previous recorded delay on the same train or route (float)
- `ROLLING_MEAN_DELAY_7D`, `ROLLING_MEAN_DELAY_30D`: Rolling mean delay values over 7/30 days
- `HOUR`: hour of day (int)
- `HOUR_SIN`, `HOUR_COS`: Cyclical encodings of hour (float)
- `DISTANCE`, `STOPS`: Numeric operational features
- `WEATHER_IMPACT`: Numeric or categorical proxy for weather contribution
- `OPERATOR_ID`, `ROUTE_ID`: Categorical features describing route or operator

> Tip: Keep this feature dictionary up-to-date when adding or removing engineered features.

In [ ]:
# ---
# Finalize best_model variable and save the full pipeline (preprocessor + model)
# This cell attempts to choose the best model variable available from earlier steps, then saves it to disk.
best_model = None
best_choice = None
for var in ['xgb_opt_pipe','xgb_opt','xgb_best','lgb_best','rf_best','rf_search','rf_pipe','linear_pipe']:
    if var in globals():
        m = globals()[var]
        if m is not None:
            best_model = m
            best_choice = var
            break

if best_model is None:
    print('No trained model found in globals(). Using linear_pipe if available.')
    best_model = globals().get('linear_pipe', None)
    best_choice = 'linear_pipe' if best_model is not None else None

print('Selected best_model:', best_choice)

# Save pipeline as joblib if a model was found
if best_model is not None:
    best_model_name = f"{best_choice}"
    model_path = os.path.join(MODEL_DIR, f"{best_model_name}_pipeline.joblib")
    try:
        joblib.dump(best_model, model_path)
        print('Saved best pipeline to', model_path)
    except Exception as e:
        print('Failed to save best pipeline:', e)
else:
    print('Skipping save; no model available.')

In [ ]:
# ---
# Final summary: standardized metrics table and saving a final CSV
try:
    if 'results_df' not in globals():
        results_df = pd.DataFrame(RESULTS)
    if 'rmse' in results_df.columns:
        results_df_sorted = results_df.sort_values('rmse')
        display(results_df_sorted)
        metrics_out_file = os.path.join(MODEL_DIR, 'final_metrics_summary.csv')
        results_df_sorted.to_csv(metrics_out_file, index=False)
        print('Saved final metrics summary to', metrics_out_file)
    else:
        print('No "rmse" column in results_df; raw RESULTS is shown:')
        display(results_df)
except Exception as e:
    print('Failed to create final summary:', e)

## How to run this notebook (Tips & Best Practices)

**Local GPU (recommended for XGBoost/LightGBM):** Install GPU drivers and ensure `nvidia-smi` is available. XGBoost uses `tree_method='gpu_hist'` when GPU detected.

**Downsample:** If running locally with limited resources, set `DOWNSAMPLE=True` and `MAX_ROWS` in the first section to quickly iterate.

**Experimentation:** Long hyperparameter tuning should be run in a compute environment or using job scheduling. Use `optuna` or `Ray Tune` for distributed tuning.

**Deployment:** Save the `best_model` as a `joblib` object in `models/` and create a simple API wrapper (Flask/FastAPI) for inference. Add monitoring for model drift and data schema changes.

# ---

## Conclusion & Next Steps

- Mô hình XGBoost (tuned) thường là lựa chọn mạnh mẽ cho bài toán này khi đo bằng RMSE. Các bước tiếp theo:
  1. Tiếp tục tuning với Optuna hoặc Bayesian optimization để tăng hiệu năng.
  2. Thêm feature importance và SHAP để giải thích những dự đoán có sai số cao.
  3. Xây dựng pipeline triển khai (API/Batch) và theo dõi chất lượng mô hình theo thời gian (drift).
  4. Đánh giá tác động của việc cắt ngọn (winsorization) và thử nghiệm dùng Huber loss hoặc Pinball loss (quantile regression) nếu cần.

- Lưu ý: Hãy kiểm tra kỹ các cột tên (SCHEDULED/ACTUAL/ID) trước khi chạy notebook trên toàn bộ dữ liệu.

Cảm ơn!

In [ ]:
# ... (previous part unchanged) ...
# simple rolling mean based on SCHEDULED_DT index
route_col = _get_route_column(train_df)
if 'SCHEDULED_DT' in train_df.columns:
    # handle rows with valid datetime only
    mask_valid_dt = train_df['SCHEDULED_DT'].notna()
    if 'TARGET' in train_df.columns and mask_valid_dt.any():
        temp = train_df.loc[mask_valid_dt].copy()
        temp = temp.sort_values(['SCHEDULED_DT'] if route_col is None else [route_col, 'SCHEDULED_DT'])
        if route_col is not None:
            temp['ROLL_MEAN_7D'] = temp.groupby(route_col)['TARGET'].transform(lambda x: x.rolling('7D').mean())
        else:
            temp['ROLL_MEAN_7D'] = temp['TARGET'].rolling('7D').mean()
        # align back
        train_df['ROLL_MEAN_7D'] = pd.NA
        train_df.loc[temp.index, 'ROLL_MEAN_7D'] = temp['ROLL_MEAN_7D']
    else:
        train_df['ROLL_MEAN_7D'] = np.nan
    train_df.reset_index(inplace=True, drop=True)

In [ ]:
# ... (previous part unchanged) ...
# simple rolling mean based on SCHEDULED_DT index
route_col = _get_route_column(df)
if 'SCHEDULED_DT' in df.columns and 'TARGET' in df.columns:
    df['ROLL_MEAN_7D'] = np.nan
    if route_col is not None:
        for name, group in df.groupby(route_col):
            try:
                sub = group.dropna(subset=['SCHEDULED_DT'])
                if sub.empty:
                    continue
                sub = sub.sort_values('SCHEDULED_DT')
                sub_index = sub.index
                sub_indexed = sub.set_index('SCHEDULED_DT')
                # compute rolling 7D per group
                sub_indexed['ROLL_MEAN_7D'] = sub_indexed['TARGET'].rolling('7D').mean()
                # map back
                df.loc[sub_index, 'ROLL_MEAN_7D'] = sub_indexed['ROLL_MEAN_7D'].values
            except Exception:
                continue
    else:
        # global rolling when no route column
        df = df.sort_values('SCHEDULED_DT')
        df.index = pd.to_datetime(df['SCHEDULED_DT'])
        df['ROLL_MEAN_7D'] = df['TARGET'].rolling('7D').mean()
        df.reset_index(drop=True, inplace=True)
else:
    df['ROLL_MEAN_7D'] = df['TARGET'].median() if 'TARGET' in df.columns else 0
# Reset index sanity
if 'SCHEDULED_DT' in df.columns and df.index.name == 'SCHEDULED_DT':
    df.reset_index(inplace=True)
# ... rest unchanged ...